# How have global energy production trends changed over time?

## Goals

By the end of this case, you should be very comfortable using `pandas` to apply functions to Series, as well with manipulating larger datasets. You'll also gain some experience with data cleaning. The `pandas` methods you'll be learning today are:

* `.copy()`
* `.drop_duplicates()`
* `.apply()`
* `.value_counts()`
* `.reset_index()`
* `.str.count()`
* `.fillna()`
* `.sum()`
* `.sort_values()`

This case also introduces the `%%time` command.


## Introduction

**Business Context.** Global electricity production, consumption, import, and export is complex and interesting for a variety of reasons. Each country has to keep track of a vast array of information to ensure that they produce enough electricity, yet balance these needs against medium-term financial implications and environmental concerns.

You are an analyst working at a non-governmental organization (NGO) that reports on global energy trends. Your department has obtained a large CSV file, but your colleagues are battling to extract relevant insights from it using Excel due to its size and format. Worse still, it has thousands of variables and they are not sure which ones are interesting. Thus, you have been made responsible for supporting your team's journalists by providing them with data and insights that they can turn into written reports.

**Business Problem.**  Your task is to **understand the information that is available and extract key insights for an upcoming report on global power patterns**. Specifically, your team wants you to answer the following four questions:

1. How much power is produced?
2. How much power is consumed?
3. How much power is imported and exported? 
4. How much of this power is renewable?

**Analytical Context.** The data is stored in a large CSV file containing information on power production and consumption by country and year. You will: 1) manipulate the data to create more categories from the existing columns, and 2) find the biggest players in different categories related to the research questions.

## Getting started with the International Energy Statistics data

The data file you have been given is a single CSV located at `data/all_energy_statistics.csv`. Your colleagues have informed you that the data is from http://data.un.org/Explorer.aspx, but they don't know much else about it. 

They specifically note that the data is very ["narrow"](https://en.wikipedia.org/wiki/Wide_and_narrow_data). Although the file contains data for a wide variety of things, such as "Total Energy Production" all the way through to "Additives and Oxygenates - Exports", it has very few columns. 

Generally, when dealing with "wide" data, we can be fairly sure that all data in the same column is comparable. In this case, you'll notice a `unit` column. Not all numerical data in the `quantity` column is directly comparable. For example, sometimes the number in this column is defined in terms of "Metric tons, thousand" and sometimes in "Kilowatt-hours, million" - evidently very different concepts!

As always, our first step is to load `pandas`, read the data from disk, and take a look at the first few rows:

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("all_energy_statistics.csv")

In [ ]:
df

You'll notice that there is more of a delay than before when running the `read_csv` function. This dataset has over 1 million rows, so it takes a while to load it all into memory. From the first rows, we can immediately gain some useful insights:

* The `category` column looks like it is well organized. All the samples we see are lowercase and underscores are used instead of spaces
* The `commodity_transaction` column looks more like a human-readable description. We can see how it includes a description of the category (e.g. "additives_and_oxygenates" matches with "Additives and Oxygenates" and "wind_electricity" matches with "Electricity - ....wind....")
* We see `year` ranges from at least 1995 to 2014 
* As mentioned before, we'll need to be careful when comparing quantities, as the `unit` column might change the meaning of the `quantity` column.

A good first question to ask is how many unique values there are for the following columns:

* `country_or_area`
* `category`

Let's find out. For that, we will use the **`.drop_duplicates()`** method, that returns the distinct values in a given Series:

In [ ]:
# country_or_area
df["country_or_area"].drop_duplicates()

In [ ]:
# category
df["category"].drop_duplicates()

We can see that `country_or_area` has 243 unique values, more than the officially recognized 195, because this list includes some former countries such as the USSR as well as areas like Antarctic Fisheries which are not formal countries. As expected, the `categories` column is well standardized and breaks each row into one of 71 unique categories.

Let's find out the year range:

In [ ]:
print(df["year"].min())
print(df["year"].max())

In terms of time, our data ranges from 1990 - 2014 inclusive, so 25 years in total.

## Using `.apply()` to loop through the rows of a Series

Since the `commodity_transaction` column is a bit chaotic, we'll need to touch it up a bit. Let's create a copy of our DataFrame with [**`.copy()`**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.copy.html) before we start changing it so we can refer back to the original values if necessary.

In [ ]:
df_orig = df.copy()

The first thing we noticed about the `commodity_transaction` column is that it uses hyphens (`-`) as separators. We can also see that it uses lowercase and capital letters - often something that makes analysis harder if we are going to do any string matching (e.g. find the word "production", which might skip descriptions which use "Production" instead). 

Let's start by lowercasing all of the descriptions. In a previous case, you learned how to do this by creating a separate list, looping through the DataFrame, and then adding all the items from the list as a new column. We could achieve what we wanted as follows:

In [ ]:
%%time
clean_transaction_list = []

for item in df['commodity_transaction']:
    item = item.lower()
    clean_transaction_list.append(item)
    
clean_series = pd.Series(clean_transaction_list)

In [ ]:
clean_series.head()

Notice that we added **`%%time`** at the top of our cell. This keyword tells Jupyter to output information about how long it took to run that cell. We can see that looping through our DataFrame and creating the Series took approximately 1 second to complete.

Because it's very common to apply the same operation on every row of a dataset, `pandas` provides a shortcut to do this. You can use the **`.apply()`** method on a DataFrame directly and pass in a function to apply it to every row. This is more efficient in two ways:

* It takes fewer lines of code, so it's faster to write the code (and to read it)
* `apply()` is optimized to take advantage of modern CPU features such as vectorization, so it runs in less time

We can achieve exactly the same result as we did with our `for` loop using the `apply()` function as follows:

In [ ]:
%%time
# We pass in the str.lower() function
# But notice we didn't include the ()
clean_series_2 = df['commodity_transaction'].apply(str.lower)

In [ ]:
clean_series_2.head()

Here we can see that `.apply()` executed twice as quickly and produced the same results (the `clean_series` and `clean_series_2` Series are the same). You can read more about the `apply()` method [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html), but in essence you call it from a column of a DataFrame and pass in a function. It then applies that function to every element of that column in the DataFrame. In this case, we passed in the `str.lower` function, which converts a string to lowercase.

We noted before that the `commodity_transaction` column seemed to use hyphens to separate different concepts in a single column. Let's do some more analysis to see if this is true across the board. It sounds like a good idea to start by finding out how many of the unique values in `commodity_transaction` contain:

- 0 hyphens
- exactly 1 hyphen
- more than 1 hyphen

This looks like a job for `pandas` **string methods**, which you can access by typing **`.str`** after the name of a `pandas` string Series. In this case, we can use the **`.str.count()`** method to count how many hyphens there are in each row:

In [ ]:
df["commodity_transaction"].drop_duplicates().str.count("-").head(10)

If we wanted to know how many rows have 0 hyphens, 1 hyphen, and more than 1 hyphen, we could use the **`.value_counts()`** method, which creates a frequency table from a `pandas` Series:

In [ ]:
df["commodity_transaction"].drop_duplicates().str.count("-").value_counts()

We can see that most descriptions have exactly one hyphen, strengthening the idea that the first part of the description before the hyphen is linked to `category`, while the rest is more descriptive. We should take a closer look at the ones with zero hyphens as there are only 57 of these.

### Exercise 1:

Write code to print out all unique descriptions with zero hyphens. What do you notice about these?

**Hint:** In this case, a good old `for` loop can be of help.

**Answer.** One possible solution is given below:

In [ ]:
for ct in df["commodity_transaction"].drop_duplicates():
    if "-" in ct:
        pass
    else:
        print(ct)

Tricky! We see an inconsistency in the data, where some descriptions use m-dashes (`–`) instead of hyphens (`-`). This is barely noticeable to a human reader, but can cause issues for computers which see the two as completely distinct characters.

### Passing our own functions to `apply()`

We previously passed the built-in `str.lower()` function to the `apply()` function to apply to it every row in our DataFrame. Now we want to clean up the m-dashes and lowercase the results at the same time. Let's write our own custom Python function to do both, and pass that to `apply()`. You can read more about writing your own custom functions in Python [here](https://www.w3schools.com/python/python_functions.asp):

In [ ]:
def clean_transaction_description(transaction_description):
    """Lowercase the input and replace all m-dashes with hyphens
    """
    clean = transaction_description.lower()
    clean = clean.replace("–", "-")
    return clean

df['clean_transaction'] = df['commodity_transaction'].apply(clean_transaction_description)
df.head()

Here we used `apply()` again, but this time passed in our own function which did both the lowercasing and the replacing of m-dashes with hyphens.

We've now seen how to use the `apply()` function with both built-in functions and our own custom functions. There's one more way we can use `apply()` though: with custom [anonymous functions](https://www.w3schools.com/python/python_lambda.asp) (a.k.a. `lambda` functions), which we have studied before. These are great if you don't want to bother with creating and naming a custom function when just declaring it on the fly will do.

### Exercise 2:

Create the `clean_transaction_lambda` column in `df`, this time using `.apply()` and an anonymous function. The resulting column should be equivalent to the `clean_transaction` column that we've just created.

**Hint:** Remember that to make a string lower case, you can use `"my_string".lower()` and to replace a character inside a string you can use `"my_string".replace("t","p")`

**Answer.**

In [ ]:
# lowercase the description and replace m-dashes with hyphens in one line
df['commodity_transaction_lambda'] = df['commodity_transaction'].apply(lambda x: x.lower().replace("–", "-"))

This code is functionally equivalent to what we ran before, but it's more concise. Instead of giving our function a name (`clean_transaction_description`), we can declare an anonymous function by using the `lambda` keyword. This says that we are going to pass in a series of `x` values (the descriptions), and describes what to do to each of them. The advantage of doing this is that it's more concise. The disadvantage is that it can be harder to read and it prevents us from using our function later without redefining it all over again.

### Extracting the most interesting rows

It's hard to manually inspect 2,000+ unique description values, but we know from our business problem that we're mainly interested in: 

* Import
* Export
* Total production
* Total demand or consumption
* Renewables

We can search for some keywords in the descriptions using code similar to the following:

In [ ]:
for ct in df["clean_transaction"].drop_duplicates():
    if "import" in ct:
        print(ct)

This gives us a much more manageable list to look through, and we can see that "electricity - imports" is likely an interesting value. We can cross-check this in the main dataset (and see all columns to boot) as follows:

In [ ]:
df[df["clean_transaction"] == "electricity - imports"].head()

### Exercise 3:

Use the above method or any other method that you prefer, explore the electricity transaction descriptions and define a Python list containing the 9 most interesting ones. These should cover the total values for import, export, total production, total demand, and renewable electricity production.

**Answer.** One possible solution is given below:

In [ ]:
# The first four values handle demand, production, import and exports
# The others are all values that match the `total ..... production` pattern except for `thermal` which 
# loosely describes all non-renewable sources of production
keep_values =  ["Electricity - Gross demand",
        "Electricity - Gross production",
        "Electricity - imports",
        "Electricity - exports",
        "Electricity - total hydro production",
        "Electricity - total wind production",
        "Electricity - total solar production",
        "Electricity - total geothermal production",
        "Electricity - total tide, wave production",
]

### Pivoting the interesting values into their own columns

Of course, now that we've identified the most interesting transaction descriptions, we probably ought to pull them out of that single column that they're stuck in. Let's pivot our data to a more useable format, keeping each of these interesting values as new columns:

In [ ]:
# we'll keep our "interesting" values after we turn them into columns
# but we'll also keep the "country" and "year" columns
final_keep_values = ["country_or_area", "year"] + keep_values

# Turn values in the 'commodity transaction' column
# into our new column names
# and keep only the 'quantity' column as the new values
df_countries = pd.pivot_table(
    df,
    values="quantity",
    index=["country_or_area", "year"],
    columns="commodity_transaction",
).reset_index()[final_keep_values]

# rename the columns to be more concise
df_countries.columns = [
    "country",
    "year",
    "demand",
    "production",
    "imports",
    "exports",
    "hydro",
    "wind",
    "solar",
    "geothermal",
    "tide",
]

# output with the energy production leaders first
df_countries.sort_values(by="production", ascending=False)

(In the code cell above we used the **`.reset_index()`** method. This method makes the index of the DataFrame a column and creates a new index that is a sequence of increasing integers).

We can see that our data is in a much more user-friendly format now. We have kept only the quantity column and each row now represents one country in a particular year. If we had data for each year for each of the 243 countries or areas, we would expect to have 6,075 rows, but we have only 5,568. This makes sense as some countries stopped existing and data collection in general has become much easier and more consistent over time. Let's take a look at how many countries we have data on for each year:

In [ ]:
df_countries['year'].value_counts()

As expected, in earlier years, we have data for fewer countries.

## Exploring growth of power production and renewables

As mentioned, the team is interested in analyzing countries based on their renewable energy production. We currently know how much power they produce in total and how much of this is due to each of a number of renewable options. We'll start by adding some supplementary data and then analyzing our dataset for interesting countries and patterns.

### Exercise 4:

Add two new summary columns called `renewable_total` and `renewable_percent`. The latter should be the percentage of total power production which is made up of renewable energy.

**Hint:** You might notice that some values are `na`, meaning "not available". We can probably assume that these are 0 (though this might not always be meaningful; e.g. if we don't have data on the USSR in 2014, it's not because its power plants are all turned off!). You can use the `pandas` [**`.fillna()`**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html) method to replace `na` values with 0. Additionally, the [**`.sum()`**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html) method can help you with finding the total production for each row (you can do that manually if you prefer as well).

**Answer.** One possible solution is given below:

In [ ]:
# replace the `na` values with 0
df_countries = df_countries.fillna(0)

# Here we use the .sum() method. axis=1 means sum horizontally, not vertically (the default)
df_countries["renewable_total"] = df_countries[["hydro", "wind", "solar", "geothermal", "tide"]].sum(axis=1)
df_countries["renewable_percent"] = df_countries["renewable_total"] / df_countries['production']
df_countries

### Exercise 5:

Considering only the most recent year that we have data for (2014), which 5 countries produced the largest proportion of their power through renewables, and which 5 countries produced the smallest proportion of their power through renewables?

**Hint:** You can use the [**`.sort_values()`**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html) method in `pandas` to sort a DataFrame by a specific column, either descending or ascending. To tell this method which column to sort by, you use the `by` argument, and to make it a Z-A sort, you add the `ascending=False` argument.

**Answer.** One possible solution is given below:

In [ ]:
df_countries[(df_countries["year"] == 2014)].sort_values(
    by="renewable_percent", ascending=False
).head(5)

We can see that Lesotho is on top, generating 100% of its power using hydro. Lesotho is a pretty tiny country though, and also imports about half as much power as it produces. The other countries on the list are also relatively small players in terms of total energy production.

In [ ]:
df_countries[(df_countries["year"] == 2014)].sort_values(
    by="renewable_percent"
).head(5)

We can see even more relatively small players in our list of countries which produce no renewable power.

### Question:

Why do you think we are seeing a lot of very small countries on both lists?

Very small countries are not particularly representative of the global renewable power situation, so your team asks you to restrict your analysis only to countries that produce a lot of power.

### Exercise 6:

Repeat the above analysis but only look at the countries in the top 10% and bottom 10% of total power production.

**Hint:** You can filter a DataFrame with multiple conditions by using the `&` symbol (similar to when writing conditional `if` statements); e.g.:

`df_countries[(df_countries.year == 2014) & (df_countries.wind > 0)]` 

would give you a DataFrame of all countries in 2014 which had produced at least some wind power.

**Answer.** One possible solution is given below:

In [ ]:
threshold = df_countries["production"].quantile(0.9)
df_countries[
    (df_countries.production > threshold) & (df_countries.year == 2014)
].sort_values(by="renewable_percent", ascending=False).head(5)

This list now has more countries on it that most people are likely to associate with renewable power! We can see that hydro and wind are popular ways of generating renewable power (by contrast our previous "top" list contained 0 wind generation).

In [ ]:
df_countries[
    (df_countries.production > threshold) & (df_countries.year == 2014)
].sort_values(by="renewable_percent").head(5)

And here we can see countries which produce a lot of power but barely any of it renewable, all in Asia or Africa.

Of course, your team is also interested in looking at change in renewable energy over time. Let's look at the top and bottom 5 countries where the percentage of renewable energy they produced in 2014 is very different from the percentage in 1990. Let's add a new column to our DataFrame to display the difference in percentage renewable energy production between 2014 and 1990. We can use the `pivot()` method again to create a DataFrame which has 1990 and 2014 as columns and `renewable_percent` as values to help with this by using the following code:

In [ ]:
# get a DataFrame with only the 1990 and 2014 values kept, and as columns
renewable_change = pd.pivot_table(
    df_countries, values="renewable_percent", index=["country"], columns="year",
).reset_index()[["country", 1990, 2014]]

# add the diff column to see the chnage
renewable_change["diff"] = renewable_change[2014] - renewable_change[1990]
renewable_change.sort_values(by="diff", ascending=False).head(5)

As before, we can see some pretty small countries. All of them went from producing zero or nearly zero renewable energy in 1990 to 40% or more by 2014. These are some great countries for the team to dig more into.

In [ ]:
renewable_change.sort_values(by="diff").head(5)

And there are some drops too. Sri Lanka was almost 100% renewable energy in 1990, but only 38% in 2014. Let's take a look at the larger ones:

In [ ]:
# get only the top producers and redo the analysis
threshold = df_countries.production.quantile(0.9)
df_countries_large = df_countries[df_countries.production > threshold]

renewable_change = pd.pivot_table(
    df_countries_large, values="renewable_percent", index=["country"], columns="year",
).reset_index()[["country", 1990, 2014]]

renewable_change["diff"] = renewable_change[2014] - renewable_change[1990]
renewable_change.sort_values(by="diff", ascending=False).head(5)

Spain, Italy and the UK have made some good progress with 10 - 20% growth in renewable power share. But considering only larger countries makes the drop-off quite dramatic after these three, with Poland and Austria rounding out the top 5 with only 4% increases each.

In [ ]:
renewable_change.sort_values(by="diff").head(5)

On the bottom of the list, we see developing countries like Brazil and India finding it hard to keep growing their renewable energy sources at the same rate as their economies.

## Largest importers and exporters of energy

The final thing that your team wants to look into is imports and exports of energy by country.

### Exercise 7:

Your team wants to know which countries have imported and exported the most power in total, the percentage of their production that is exported, and the percentage of their demand that is imported. Write some code to answer these questions.

**Answer.** One possible solution is given below:

In [ ]:
df_countries_agg = df_countries.groupby("country").sum()

In [ ]:
df_countries_agg["imports"].sort_values(ascending=False).head()

The United States imports a lot of energy. We can guess that a lot of this comes from Canada, although the dataset doesn't actually contain this data.

In [ ]:
df_countries_agg["exports"].sort_values(ascending=False).head()

France, Germany, and Canada export a lot of energy! Interestingly, Germany and Switzerland are on both lists, importing and exporting power. This is perhaps because Switzerland's large amount of hydro power is difficult to store.

Let's now look at percentages:

In [ ]:
df_countries_agg['percent_export'] = df_countries_agg['exports'] / df_countries_agg['production']
df_countries_agg['percent_import'] = df_countries_agg['imports'] / df_countries_agg['demand']

In [ ]:
df_countries_agg.sort_values(by='percent_export', ascending=False).head(5)

There are some smaller countries that export over 50% of the power that they produce. Let's look again with the 10% threshold applied:

In [ ]:
threshold = df_countries_agg["production"].quantile(0.9)
df_countries_agg[df_countries_agg.production > threshold].sort_values(
    by="percent_export", ascending=False
).head(5)

Larger countries export no more than 12% of their produced power.

In [ ]:
df_countries_agg.sort_values(by='percent_import', ascending=False).head(5)

For import percentage, we again see small countries feature most. Luxembourg imports more power than it produces! Let's add the 10% threshold back:

In [ ]:
df_countries_agg[df_countries_agg.production > threshold].sort_values(
    by="percent_import", ascending=False
).head(5)

We can see that Italy is hugely dependent on its neighbors, being on the leader list for both total quantity of imported energy and as a percentage of demand.

## Conclusions


We saw a number of interesting trends in the global energy industry. Specifically, we saw that many countries are relying more and more on renewables, but that some of the countries with fast-growing demand are forced to turn to non-renewable sources to keep up.

We also noticed that contrary to our expectations of some countries being "net importers" and others being "net exporters" of power, many countries actually both import *and* export large amounts of power.

## Takeaways

In this case, we covered some more features of `pandas` and got more practice with the features we covered previously. Specifically we saw how to:

* Use the `apply()` method in `pandas` with built-in functions, custom functions, and anonymous functions
* Work with large datasets and explore these using basic string matching to find interesting columns, and reformat the results into more convenient formats
* Pivot between wide and narrow formats
* Do some basic data cleaning

While you'll learn more advanced functionality than this in later cases, these basics will be used again and again, so keep coming back to this case to reference material as often as you need.

## Attribution

"Energy Statistics Database", UN Data, [UN Data license](http://data.un.org/Host.aspx?Content=UNdataUse), http://data.un.org/Explorer.aspx